In [1]:
%cd improved_summac

D:\msc_work\sumVeracity\improved_summac


In [2]:
from improved_summac.benchmark import SummaCBenchmark
import transformers
import sklearn, pandas as pd, seaborn as sns
from improved_summac.model_summac import SummaCZS
from improved_summac.utils_scoring import ScorerWrapper
from improved_summac.utils_scorer import compute_doc_level
import warnings
warnings.filterwarnings("ignore")

import os
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"
os.environ["TRANSFORMERS_VERBOSITY"] = "error"
transformers.utils.logging.set_verbosity(transformers.logging.ERROR)
print(transformers.utils.logging.get_verbosity())

benchmark = SummaCBenchmark(benchmark_folder="data/summac_benchmark/", cut="test")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aviv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


40


# Table 2: Main Table of Results


In [3]:
use_cache = True
scorers = [
    {
        "name": "NLI_Baseline",
        "model": SummaCZS(nli_granularity="sentence", model_name="vitc", imager_load_cache=use_cache, use_nli=True, use_sts=False, use_mlm=False),
        "sign": 1,
    },

    {
        "name": "STS_SENTENCE",
        "model": SummaCZS(sts_granularity="sentence", model_name="vitc", imager_load_cache=use_cache, use_nli=False, use_sts=True, use_mlm=False),
        "sign": 1,
    },
    {
        "name": "STS_PARAGRAPH",
        "model": SummaCZS(sts_granularity="paragraph", model_name="vitc", imager_load_cache=use_cache, use_nli=False, use_sts=True, use_mlm=False),
        "sign": 1,
    },

    {
        "name": "MLM_PARAGRAPH",
        "model": SummaCZS(mlm_granularity="paragraph", model_name="vitc", imager_load_cache=use_cache, use_nli=False, use_sts=False, use_mlm=True),
        "sign": 1,
    },
    {
        "name": "MLM_DOCUMENT",
        "model": SummaCZS(mlm_granularity="document", model_name="vitc", imager_load_cache=use_cache, use_nli=False, use_sts=False, use_mlm=True),
        "sign": 1,
    },

    {
        "name": "NLI_STS_SENTENCE",
        "model": SummaCZS(nli_granularity="sentence", sts_granularity="sentence", model_name="vitc", imager_load_cache=use_cache, use_nli=True, use_sts=True, use_mlm=False),
        "sign": 1,
    },
    {
        "name": "NLI_STS_PARAGRAPH",
        "model": SummaCZS(nli_granularity="sentence", sts_granularity="paragraph", model_name="vitc", imager_load_cache=use_cache, use_nli=True, use_sts=True, use_mlm=False),
        "sign": 1,
    },

    {
        "name": "NLI_MLM_PARAGRAPH",
        "model": SummaCZS(nli_granularity="sentence", mlm_granularity="sentence", model_name="vitc", imager_load_cache=use_cache, use_nli=True, use_sts=False, use_mlm=True),
        "sign": 1,
    },
    {
        "name": "NLI_MLM_DOCUMENT",
        "model": SummaCZS(nli_granularity="sentence", mlm_granularity="document", model_name="vitc", imager_load_cache=use_cache, use_nli=True, use_sts=False, use_mlm=True),
        "sign": 1,
    },

    {
        "name": "STS_SENTENCE_MLM_PARAGRAPH",
        "model": SummaCZS(sts_granularity="sentence", mlm_granularity="paragraph", model_name="vitc", imager_load_cache=use_cache, use_nli=False, use_sts=True, use_mlm=True),
        "sign": 1,
    },
    {
        "name": "STS_SENTENCE_MLM_DOCUMENT",
        "model": SummaCZS(sts_granularity="sentence", mlm_granularity="document", model_name="vitc", imager_load_cache=use_cache, use_nli=False, use_sts=True, use_mlm=True),
        "sign": 1,
    },

    {
        "name": "STS_PARAGRAPH_MLM_PARAGRAPH",
        "model": SummaCZS(sts_granularity="paragraph", mlm_granularity="paragraph", model_name="vitc", imager_load_cache=use_cache, use_nli=False, use_sts=True, use_mlm=True),
        "sign": 1,
    },
    {
        "name": "STS_PARAGRAPH_MLM_DOCUMENT",
        "model": SummaCZS(sts_granularity="paragraph", mlm_granularity="document", model_name="vitc", imager_load_cache=use_cache, use_nli=False, use_sts=True, use_mlm=True),
        "sign": 1,
    },

    {
        "name": "NLI_STS_SENTENCE_MLM_PARAGRAPH",
        "model": SummaCZS(nli_granularity="sentence", sts_granularity="sentence", mlm_granularity="paragraph", model_name="vitc", imager_load_cache=use_cache, use_nli=True, use_sts=True, use_mlm=True),
        "sign": 1,
    },
    {
        "name": "NLI_STS_PARAGRAPH_MLM_PARAGRAPH",
        "model": SummaCZS(nli_granularity="sentence", sts_granularity="paragraph", mlm_granularity="paragraph", model_name="vitc", imager_load_cache=use_cache, use_nli=True, use_sts=True, use_mlm=True),
        "sign": 1,
    },
    {
        "name": "NLI_STS_PARAGRAPH_MLM_DOCUMENT",
        "model": SummaCZS(nli_granularity="sentence", sts_granularity="paragraph", mlm_granularity="document", model_name="vitc", imager_load_cache=use_cache, use_nli=True, use_sts=True, use_mlm=True),
        "sign": 1,
    },
    {
        "name": "NLI_STS_SENTENCE_MLM_DOCUMENT",
        "model": SummaCZS(nli_granularity="sentence", sts_granularity="sentence", mlm_granularity="document", model_name="vitc", imager_load_cache=use_cache, use_nli=True, use_sts=True, use_mlm=True),
        "sign": 1,
    },
]

scorer_doc = ScorerWrapper([scorers[-1]], scoring_method="sum", max_batch_size=50, use_caching=True)

In [4]:
results = []
for dataset in benchmark.datasets:
	print("======= %s ========" % (dataset["name"]))
	datas = dataset["dataset"][:30]
	compute_doc_level(scorer_doc, datas)

	labels = [d["label"] for d in datas]
	pred_labels = [k for k in datas[0].keys() if "pred_" in k]
	for pred_label in pred_labels:
		model_name, input_type = pred_label.replace("pred_", "").split("|")
		preds = [d[pred_label] for d in datas]
		scores = [d[pred_label.replace("pred_", "")] for d in datas]
		balanced_acc = sklearn.metrics.balanced_accuracy_score(labels, preds)
		roc_auc = sklearn.metrics.roc_auc_score(labels, scores)

		results.append({"model_name": model_name, "dataset_name": dataset["name"],
						"input": input_type, "%s_bacc" % (dataset["name"]): balanced_acc,
						"%s_roc_auc" % (dataset["name"]): roc_auc,
						"labels": labels, "preds": preds, "scores": scores})

In [5]:
pd.DataFrame(results).to_parquet('results/results_big_new.parquet.gzip', compression='gzip')

## Balanced accuracy score

In [5]:
cm = sns.light_palette("green", as_cmap=True)

def highlight_max(data):
	is_max = data == data.max()
	return ['font-weight: bold' if v else '' for v in is_max]

# df = pd.DataFrame(results)
df = pd.read_parquet('results/results_big_new.parquet.gzip')
df = df.groupby(["model_name", "input"]).agg({"%s_bacc" % (d): "mean" for d in ["xsumfaith", "polytope", "factcc", "summeval", "frank"]})
df.rename(columns={k: k.replace("_bacc", "") for k in df.keys()}, inplace=True)
df.drop("total", inplace=True)

df["overall"] = (df["factcc"] + df["frank"]  + df["polytope"] + df["summeval"] + df["xsumfaith"]) / (
	5.0)

bacc = df.style.apply(highlight_max).background_gradient(cmap=cm, high=1.0, low=0.0).set_precision(3).set_caption(
	"Balanced Accuracy")

In [6]:
bacc

,,xsumfaith,polytope,factcc,summeval,frank,overall
model_name,input,,,,,,
MLM_DOCUMENT,doc,0.555,0.685,0.573,0.666,0.785,0.653
MLM_PARAGRAPH,doc,0.587,0.589,0.573,0.666,0.785,0.640
NLI_Baseline,doc,0.666,0.612,0.902,0.818,0.803,0.760
NLI_MLM_DOCUMENT,doc,0.643,0.680,0.888,0.823,0.811,0.769
NLI_MLM_PARAGRAPH,doc,0.641,0.576,0.821,0.822,0.815,0.735
NLI_STS_PARAGRAPH,doc,0.632,0.587,0.861,0.795,0.794,0.734
NLI_STS_PARAGRAPH_MLM_DOCUMENT,doc,0.626,0.617,0.852,0.799,0.806,0.740
NLI_STS_PARAGRAPH_MLM_PARAGRAPH,doc,0.646,0.577,0.852,0.799,0.806,0.736
NLI_STS_SENTENCE,doc,0.655,0.572,0.809,0.755,0.787,0.715


## ROC AUC score

In [7]:
# df = pd.DataFrame(results)
df = pd.read_parquet('results/results_big_new.parquet.gzip')
df = df.groupby(["model_name", "input"]).agg({"%s_roc_auc" % (d): "mean" for d in ["xsumfaith", "polytope", "factcc", "summeval", "frank"]})
df.rename(columns={k: k.replace("_roc_auc", "") for k in df.keys()}, inplace=True)
df.drop("total", inplace=True)

df["overall"] = (df["factcc"] + df["frank"] + df["polytope"] + df["summeval"] + df["xsumfaith"]) / (
	5.0)

roc = df.style.apply(highlight_max).background_gradient(cmap=cm, high=1.0, low=0.0).set_precision(3).set_caption("ROC AUC")

In [8]:
roc

,,xsumfaith,polytope,factcc,summeval,frank,overall
model_name,input,,,,,,
MLM_DOCUMENT,doc,0.532,0.703,0.564,0.685,0.834,0.663
MLM_PARAGRAPH,doc,0.601,0.515,0.564,0.685,0.834,0.640
NLI_Baseline,doc,0.708,0.584,0.937,0.882,0.886,0.799
NLI_MLM_DOCUMENT,doc,0.685,0.683,0.908,0.861,0.890,0.805
NLI_MLM_PARAGRAPH,doc,0.683,0.522,0.859,0.860,0.877,0.760
NLI_STS_PARAGRAPH,doc,0.676,0.559,0.877,0.813,0.859,0.757
NLI_STS_PARAGRAPH_MLM_DOCUMENT,doc,0.676,0.604,0.878,0.822,0.872,0.770
NLI_STS_PARAGRAPH_MLM_PARAGRAPH,doc,0.689,0.537,0.878,0.822,0.872,0.759
NLI_STS_SENTENCE,doc,0.680,0.545,0.874,0.825,0.843,0.753
